In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import glob
from tensorflow.keras import layers

In [2]:
files = np.sort(glob.glob("./data/*"))
label_classes = ["talking", "non"]
batch_size = 32

In [3]:
def initial_process(files, label_classes):
    for file in files:
        data = []
        labels = []
        temp = np.clip(pd.read_csv(open(file, "r")).to_numpy(), 1, 40)
        data.append(temp)
        for j in range(len(label_classes)):
            if label_classes[j] in file:
                labels.append(j)
        yield data, labels

In [4]:
data = initial_process
(files, label_classes)
dataset = tf.data.Dataset.from_generator(initial_process, args=[files, label_classes], output_signature=(tf.TensorSpec(shape=(1, 50, 1), dtype=tf.float32), tf.TensorSpec(shape=(1,), dtype=tf.int32)))
dataset = dataset.repeat(100)
dataset = dataset.shuffle(len(list(dataset)))
                         

In [5]:
train_ds = dataset.take(int(0.8 * len(list(dataset))))
val_ds = dataset.skip(int(0.8 * len(list(dataset))))

In [6]:
def squeeze_data(data, label):
    data = tf.squeeze(data)
    tf.reshape(data, [51])
    return data, tf.squeeze(label)

In [7]:
'''from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(1,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])

model.summary()
'''

"from tensorflow.keras import layers\n\nmodel = tf.keras.Sequential([\n    layers.Dense(32, activation='relu', input_shape=(1,)),\n    layers.Dense(64, activation='relu'),\n    layers.Dense(1, activation='linear')\n])\n\nmodel.summary()\n"

In [8]:
model = tf.keras.Sequential([
    layers.Conv1D(32, 3, activation="relu", input_shape=(51,1)),
    layers.MaxPool1D(2),
    layers.Conv1D(64, 3, activation="relu"),
    layers.MaxPool1D(2),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(2, activation="softmax")
])
model.summary()

C:\Users\Streamer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 49, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 24, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 22, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 704)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        45,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,098 (211.32 KB)

 Trainable params: 54,098 (211.32 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
model.fit(train_ds, validation_data=val_ds, epochs=10, batch_size=batch_size)

Epoch 1/10
  15997/Unknown 31s 1ms/step - accuracy: 0.6117 - loss: 0.6056

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


16000/16000 ━━━━━━━━━━━━━━━━━━━━ 46s 2ms/step - accuracy: 0.6117 - loss: 0.6056 - val_accuracy: 0.9038 - val_loss: 0.2071
Epoch 2/10


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


15991/16000 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9242 - loss: 0.1812

...

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


15974/16000 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9916 - loss: 0.0489

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


16000/16000 ━━━━━━━━━━━━━━━━━━━━ 46s 2ms/step - accuracy: 0.9916 - loss: 0.0489 - val_accuracy: 0.9998 - val_loss: 3.3080e-06


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [11]:
model.save("model.keras")